# Pipeline de Geração de Observações - Tic Tac Toe
**Author**: Gabriela Dellamora Paim, Bruno Carlan

**Version**: 12/04/2025

**Python Ver**: 3.12.9

## Checklist para geração das 64 observações de tabuleiros *in_progress*

### Distribuição de jogadas
- [ ] Seguir essa proporção:
  - 1 jogada: 4 observações  
  - 2 jogadas: 4 observações  
  - 3 jogadas: 8 observações  
  - 4 jogadas: 8 observações  
  - 5 jogadas: 12 observações  
  - 6 jogadas: 12 observações  
  - 7 jogadas: 16 observações  
- Total: **64 observações**

### Ordem dos jogadores
- [ ] Todas as partidas devem começar com **X**
- [ ] Validar que o número de jogadas de X é igual ou 1 a mais que o de O
- [ ] Determinar corretamente o jogador da vez com base no número de jogadas:
  - Número par → próximo jogador é X
  - Número ímpar → próximo jogador é O

### Validação do estado do jogo
- [ ] Nenhum estado pode conter um jogo já vencido
- [ ] Nenhum tabuleiro pode conter jogadas inválidas (ex: duas peças na mesma posição)
- [ ] Nenhum estado pode ter mais de 9 jogadas

### Diversidade de posições
- [ ] Evitar repetição de estados (usar hash ou set para controle)
- [ ] Garantir distribuição equilibrada entre as 9 posições do tabuleiro
- [ ] (Opcional) Limitar frequência de ocupação por posição (ex: máximo 50% em cada célula)

### Formato e reprodutibilidade
- [ ] Exportar as observações no formato combinado (array, dataframe, JSON, etc.)
- [ ] Incluir metadados por observação:
  - Número de jogadas  
  - Jogador da vez  
  - ID único (ou hash)
- [ ] Fixar seed aleatória (`random.seed(42)` ou equivalente) para reprodutibilidade
- [ ] (Opcional) Logar motivo de descarte de observações (ex: estado inválido, repetido, etc.)


In [33]:
import pandas as pd
import random
from collections import defaultdict
from math import ceil

PATH_OLD = './data_processed.csv'
PATH_NEW = './data.csv'
PATH_ONGOING = './data_ongoing.csv'
X_WIN   = X = '1'
O_WIN   = O ='-1'
DRAW    = '0'
ONGOING = '0.5'

random.seed(42)

In [34]:
df_processed = pd.read_csv(PATH_OLD, index_col=False)
df_processed.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', 'category']
df_processed.describe()

,0,1,2,3,4,5,6,7,8,category
count,1914.000000,1914.00000,1914.000000,1914.00000,1914.000000,1914.00000,1914.000000,1914.00000,1914.000000,1914.000000
mean,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000
std,0.886679,0.85974,0.886679,0.85974,0.912823,0.85974,0.886679,0.85974,0.886679,0.991864
min,-1.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.000000
25%,-1.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.00000,-1.000000,-1.000000
50%,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000
75%,1.000000,1.00000,1.000000,1.00000,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000
max,1.000000,1.00000,1.000000,1.00000,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000


# Gerar dados Ongoing. Utilizar um critério de observação adequado

In [ ]:
class Board:
    '''
    Classe que gerencia as regras e o estado de um tabuleiro do jogo da velha (Tic Tac Toe).

    Permite simulações de jogadas, geração de observações para modelos de IA, e controle
    de status do jogo (vitória, empate ou andamento).

    A movimentação é pseudo-aleatória: a cada chamada de `move`, uma posição disponível é escolhida aleatoriamente.
    '''
    def __init__(self, first_player=X, second_player=O):
        '''
        Inicializa o tabuleiro com jogadores definidos e espaços vazios.

        Parâmetros:
        ----------
        first_player : int
            Valor que representa o jogador que inicia a partida (X ou O).

        second_player : int
            Valor que representa o segundo jogador.
        '''
        self.players = [first_player, second_player]
        self.board = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        self.moves = [(i, j) for i in range(3) for j in range(3)]
        self.status = ONGOING
        self.moves_left = self.get_moves_left()

    def move(self, symbol: int):
        '''
        Realiza uma jogada aleatória no tabuleiro com o símbolo fornecido.

        Parâmetros:
        ----------
        symbol : int
            Representa o jogador (X ou O) que está fazendo a jogada.
        '''
        x, y = self._get_coordinates()
        if (x >= 0) and (y >= 0):
            self.board[x][y] = symbol
        self._update_status()

    def get_moves_left(self):
        '''
        Retorna a quantidade de jogadas restantes no tabuleiro.
        '''
        return len(self.moves)

    def _update_status(self):
        '''
        Atualiza o atributo `status` com base no estado atual do tabuleiro.
        '''
        self.status = self._check_wins()

    def _check_wins(self):
        '''
        Verifica se há um vencedor no tabuleiro ou se o jogo continua.

        Retorno:
        -------
        int
            Retorna o símbolo do vencedor (X ou O), ou ONGOING se o jogo estiver em andamento.
        '''
        # Diagonais
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            return self.board[1][1]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != 0:
            return self.board[1][1]
        # Linhas e colunas
        for i in range(3):
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != 0:
                return self.board[0][i]
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                return self.board[i][0]
        return ONGOING

    def _get_coordinates(self) -> tuple:
        '''
        Retorna uma coordenada aleatória ainda disponível para jogar.

        Remove a coordenada da lista de jogadas possíveis para evitar repetições.

        Retorno:
        -------
        tuple
            Par (x, y) com coordenadas da jogada. Retorna (-1, -1) se o tabuleiro estiver cheio.
        '''
        if not self.is_full():
            return self.moves.pop(random.randint(0, len(self.moves) - 1))
        return -1, -1

    def is_full(self) -> bool:
        '''
        Verifica se o tabuleiro está totalmente preenchido.

        Retorno:
        -------
        bool
            True se não há mais jogadas disponíveis, False caso contrário.
        '''
        return self.get_moves_left() <= 0

    def count_moves(self):
        '''
        Conta quantas jogadas foram feitas no tabuleiro.

        Retorno:
        -------
        int
            Quantidade total de jogadas (X ou O) feitas até o momento.
        '''
        flat = [cell for row in self.board for cell in row]
        return flat.count(O) + flat.count(X)

    def board_hash(self):
        '''
        Gera uma representação única do tabuleiro atual para evitar duplicatas.

        Retorno:
        -------
        tuple
            Hash do tabuleiro em formato 1D.
        '''
        return tuple(cell for row in self.board for cell in row)

    def next_player(self):
        '''
        Determina qual jogador deve jogar na próxima jogada.

        Retorno:
        -------
        int
            Valor do jogador atual (X ou O), com base na paridade das jogadas restantes.
        '''
        return self.players[1] if self.get_moves_left() % 2 == 0 else self.players[0]

    def get_observation(self):
        '''
        Retorna o estado atual do tabuleiro como uma lista plana 1D.

        Inclui o status atual (vitória, empate ou andamento) no final.

        Retorno:
        -------
        list
            Estado atual do tabuleiro (1D) + status.
        '''
        flat = [cell for row in self.board for cell in row]
        flat.append(self.status)
        return flat


In [43]:
@staticmethod
def generate_ongoing_boards(quantity: int = 64, category_quantity: dict = {}):
    '''
    Gera um conjunto de observações de jogos em andamento (in_progress) para o jogo da velha.

    Esta função simula jogos parciais a partir de um tabuleiro vazio, respeitando regras do jogo
    e critérios de balanceamento definidos em `category_quantity`, como a quantidade de observações
    desejadas por número de jogadas. Apenas estados válidos, únicos e em andamento são considerados.

    Parâmetros:
    ----------
    quantity : int
        Quantidade total de observações desejadas (opcional, não usado diretamente se `category_quantity` estiver definido).

    category_quantity : dict
        Dicionário que define a quantidade de observações desejadas para cada número de jogadas.
        Exemplo: {1: 4, 2: 4, 3: 8, 4: 8, 5: 12, 6: 12, 7: 16}

    Retorno:
    -------
    pd.DataFrame
        DataFrame contendo os estados gerados com as seguintes colunas:
        ['0','1','2','3','4','5','6','7','8','n_jogadas','jogador_vez','id_hash']
    '''

    generated = defaultdict(list)  # Armazena listas de observações por número de jogadas
    seen_hashes = set()            # Conjunto para evitar duplicatas

    # Gera até atingir a quantidade necessária por categoria
    while any(len(generated[k]) < v for k, v in category_quantity.items()):
        board = Board(first_player=X, second_player=O)

        # Simula uma partida até que o jogo acabe ou atinja um estado intermediário válido
        while board.status == ONGOING and board.moves_left > 0:
            symbol = board.next_player()
            board.move(symbol)

            # Se o jogo acabou nessa jogada, descartamos
            if board.status != ONGOING:
                break

            # Verifica quantidade de jogadas feitas
            flat_board = board.get_observation()[:-1]
            current_moves = board.count_moves()

            # Verifica se ainda podemos adicionar uma nova observação nessa categoria
            if current_moves in category_quantity and len(generated[current_moves]) < category_quantity[current_moves]:
                h = board.board_hash()
                if h not in seen_hashes:
                    seen_hashes.add(h)
                    jogador_da_vez = board.next_player()
                    # Salva: estado do tabuleiro (1D), número de jogadas, jogador da vez, e hash
                    generated[current_moves].append(flat_board + [current_moves, jogador_da_vez, str(h)])

    # Junta todas as observações geradas num único DataFrame
    all_obs = []
    for lst in generated.values():
        all_obs.extend(lst)

    columns = ['0','1','2','3','4','5','6','7','8','n_jogadas','jogador_vez','id_hash']
    df_final = pd.DataFrame(all_obs, columns=columns)
    return df_final


In [44]:

quantity = 400
category_quantity = {
    1 : 28,
    2 : 28,
    3 : 48,
    4 : 48,
    5 : 72,
    6 : 72,
    7 : 104
}

df_ongoing = generate_ongoing_boards(quantity, category_quantity)

df_ongoing.to_csv(PATH_ONGOING, index=False)

KeyboardInterrupt: 

In [5]:
# concatena os datasets para termos o nosso dataset tratado
df_ongoing.drop(columns=['n_jogadas', 'jogador_vez', 'id_hash'], inplace=True)
df = pd.concat([df_processed, df_ongoing])
df.to_csv(PATH_NEW, index=False)